In [1]:
from os import listdir
import numpy as np
import pyxdf
import mne

In [2]:
# Split the two streams:
def split_streams(streams):
    # Check if there are two streams:
    assert len(streams) == 2
        
    for s in streams:
        if s['info']['type'][0] == 'EEG':
            eeg_stream = s
        elif s['info']['type'][0] == 'Marker':
            marker_stream = s
    
    return eeg_stream, marker_stream

In [9]:
path = 'D:/Diplomarbeit_data/sub-P001/ses-S001/eeg'

# List files in folder:
files = [f for f in listdir(path)]


fname = path + '/' + files[0]
streams, header = pyxdf.load_xdf(fname)
    
# Split the streams:
eeg_stream, marker_stream = split_streams(streams)

eeg = eeg_stream['time_series'].T
assert eeg.shape[0] == 65
eeg_ts = eeg_stream['time_stamps']

markers = marker_stream['time_series']
# Convert list of list of strings to list of strings:
markers = [''.join(element) for element in markers]

# Convert marker strings to integers:
classes = ['LTR-s', 'LTR-l','RTL-s', 'RTL-l', 'TTB-s', 'TTB-l', 'BTT-s', 'BTT-l']
classes_map = [1, 2, 3, 4, 5, 6, 7, 8]
timings = ['Start', 'Cue', 'Break']
timings_map = [11, 12, 13]
responses = ['c c 0', 'c c 1', 'c r 0', 'c r 1', 'c l 0', 'c l 1', 'c b 0', 'c b 1', 'c t 0', 'c t 1']
responses_map = [21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

concat_codes = classes + timings + responses
concat_mappings = classes_map + timings_map + responses_map

coded_markers = markers.copy()

for i,marker in enumerate(markers):
    for j,code in enumerate(concat_codes):
        if code in marker:
            coded_markers[i] = concat_mappings[j]
            
markers_ts = marker_stream['time_stamps']

# Make Nan array with len(eeg)
aligned_markers = np.zeros(eeg_ts.shape)

# Place markers string at the align array where first time markers_ts <= eeg_ts:
for i,marker in enumerate(coded_markers):
    ts = markers_ts[i]
    idx = np.where(ts <= eeg_ts)[0][0]
    aligned_markers[idx] = marker
    

# Replace last row (unused Brainproduct markers) with the aligned recorded markers:
eeg[64,:] = aligned_markers

fs = float(eeg_stream['info']['nominal_srate'][0])

# Extract channel names:
chn_names = [eeg_stream['info']['desc'][0]['channels'][0]['channel'][i]['label'][0] for i in range(64)]
chn_names.append('Markers')
chn_labels = ['eeg' for i in range(64)]
chn_labels.append('markers')


In [10]:
info = mne.create_info(chn_names, fs)


In [11]:
raw = mne.io.RawArray(eeg, info)
raw.plot(scalings=dict(eeg=100e-6), duration=1, start=14)

Creating RawArray with float64 data, n_channels=65, n_times=93600
    Range : 0 ... 93599 =      0.000 ...   467.995 secs
Ready.
Using qt as 2D backend.
Opening raw-browser...


In [7]:
raw.info

<Info | 7 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F7, F3, Fz, F4, F8, FC5, FC1, FC2, FC6, T7, C3, Cz, ...
 chs: 64 misc
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 100.0 Hz
 meas_date: unspecified
 nchan: 64
 projs: []
 sfreq: 200.0 Hz
>

In [11]:
eeg_ts.shape

(93600,)